# Classification de locuteur

#### Objectif : créer une chaine de traitement des données textuelles sur la classification de locuteur

#### Jeu de données : citations Chirac Mitterand

* Analyses obligatoires
    * Comparer les performances avec différents pré-traitements
        * e.g Taille de vocabulaire, unigram/bigram, Stemming, ...
    * Implémenter un post-traitement sur les données Chirac/Mitterrand
    * Appliquer les traitements optimaux sur les données de test et sauver les résultats dans un fichier txt

* Compléments optionnels
    * Analyser les performances avec Word2Vec, en utilisant des stratégies d'agrégation naïves

* Ecrire un rapport succinct
    * Présentant les courbes de performances pour les paramètres les plus influents/marquants
    * Quelques conclusions sur le travail effectué

* Soumettre par mail:
    * Rapport, Notebook(s), 2 fichiers de scores (locuteur/opinion)

## Import des librairies

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import codecs
import re

## Chargement des données Mitterand / Chirac

In [14]:
# Chargement des données:
def load_pres(fname):
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour régler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
        if lab.count('M') >0:
            alllabs.append(-1)
        else: 
            alllabs.append(1)
        alltxts.append(txt)
    return alltxts,alllabs

In [15]:
path = "data/AFDpresidentutf8/corpus.tache1.learn.utf8"

alltxts,alllabs = load_pres(path)

### Verification de l'équilibre du plan d'expérience

In [16]:
C, M = np.unique(alllabs, return_counts=True)[1]
print(f"Il y a {M/(C+M) * 100} % de citations attribuées à Mitterand et {C/(C+M) * 100} % de citations attribuées à Chirac ")

Il y a 86.89669587027329 % de citations attribuées à Mitterand et 13.103304129726718 % de citations attribuées à Chirac 


On observe que le plan d'expérience n'est pas équilibré. Il va falloir échantilloner notre dataset pour se ramener à une situation à l'équilibre (50% des exemples associés à chaque locuteur).

### Under-sampling dans la classe majoritaire 'Mitterand'

In [17]:
from imblearn.under_sampling import RandomUnderSampler

# Instancier le RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='auto')  # 'auto' signifie que la stratégie est de rééquilibrer toutes les classes à la taille de la classe minoritaire.

# Appliquer la méthode fit_resample
alltxts_res, alllabs_res = rus.fit_resample(np.array(alltxts).reshape(-1, 1), alllabs)

In [18]:
C, M = np.unique(alllabs_res, return_counts=True)[1]
print(f"Il y a {M/(C+M) * 100} % de citations attribuées à Mitterand et {C/(C+M) * 100} % de citations attribuées à Chirac ")

Il y a 50.0 % de citations attribuées à Mitterand et 50.0 % de citations attribuées à Chirac 


L'under sampling a permis le réequilibrage des classes.

In [19]:
# On réassigne les variables pour plus de clarté dans le code
alltxts, alllabs = alltxts_res, alllabs_res

## Analyse des mots fréquents

In [20]:
import nltk
import string

from nltk import bigrams, trigrams, FreqDist, word_tokenize
from nltk.corpus import stopwords

# Nécessaire pour la première exécution
nltk.download("punkt")
nltk.download("stopwords")

all_texts, all_labels = load_pres(path)

text = " ".join([str(elem) for elem in all_texts])

# Retire la ponctuation
translator = str.maketrans("", "", string.punctuation)
text = text.translate(translator)

tokens = word_tokenize(text)

# Filtre les stopwords
stop_words = set(stopwords.words("english"))
tokens = list(filter(lambda token: token not in stop_words, tokens))

freq_dist = FreqDist(tokens)
vocab_size = len(freq_dist)
print(f"Taille du vocabulaire : {vocab_size}")

# Calcule les 100 bigrammes et trigrammes les plus fréquents
bigrams = list(bigrams(tokens))
trigrams = list(trigrams(tokens))

bigram_freq = FreqDist(bigrams)
trigram_freq = FreqDist(trigrams)

top_100_bigrams = bigram_freq.most_common(100)
top_100_trigrams = trigram_freq.most_common(100)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ambroisebertin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ambroisebertin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Taille du vocabulaire : 36791


In [ ]:
plt.figure(figsize=(15, 5))
freq_dist.plot(100)
plt.show()

plt.figure(figsize=(15, 5))
bigram_freq.plot(100)
plt.show()

plt.figure(figsize=(15, 5))
trigram_freq.plot(100)
plt.show()

# NB : modifier le code pour qu'il sauvegarde les figures dans un dossier "figures" (créé au préalable)

## Apprentissage du modèle

In [23]:
from sklearn.model_selection import train_test_split

alltxts, alllabs = load_pres(path)

# Diviser le dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(all_texts, all_labels, test_size=0.2, random_state=42)

print("Taille de l'ensemble d'entraînement :", len(train_texts))
print("Taille de l'ensemble de test :", len(test_texts))

Taille de l'ensemble d'entraînement : 45930
Taille de l'ensemble de test : 11483


In [25]:
# Appliquer la classification naïve bayésienne
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Créer le vectorizer
vectorizer = CountVectorizer()

# Vectoriser les données d'entraînement
train_vectors = vectorizer.fit_transform(train_texts)

# Vectoriser les données de test
test_vectors = vectorizer.transform(test_texts)

# Créer le classifieur
clf = MultinomialNB()

# Entraîner le classifieur
clf.fit(train_vectors, train_labels)

# Prédire les labels des données de test
pred = clf.predict(test_vectors)

# Calculer la précision
from sklearn.metrics import accuracy_score
print(f"Précision : {accuracy_score(test_labels, pred)}")

Précision : 0.8965427153182967
